In [7]:
from haystack import Pipeline, Document
from haystack.components.converters import TextFileToDocument
from haystack.components.writers import DocumentWriter
from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
from pathlib import Path
file_path = Path("Aditya_train.txt")
absolute_file_path = file_path.resolve()
chroma_store = ChromaDocumentStore()
# Resolve the absolute path
absolute_file_path = file_path.resolve()
print(absolute_file_path)
pipeline = Pipeline()
pipeline.add_component("converter", TextFileToDocument())
pipeline.add_component("writer_chroma", DocumentWriter(document_store=chroma_store))
pipeline.connect("converter","writer_chroma")
pipeline.run(
    {"converter": {"sources": [str(absolute_file_path)]}}
)

/unity/f2/asugandhi/Downloads/LLM_Playground/Aditya_train.txt


{'writer_chroma': {'documents_written': None}}

In [8]:
querying = Pipeline()
querying.add_component("retriever", ChromaQueryTextRetriever(chroma_store))
results = querying.run({"retriever": {"query": "SAS", "top_k": 1}})
print(results['retriever']['documents'][0])

Document(id=4131426fc6c30a306d6379a1d040425f2998e701287f2c255d53d6ad03d4c95d, content: 'Aditya Sugandhi's journey as a Software Engineer is characterized by a deep commitment to excellence...', meta: {'file_path': '/unity/f2/asugandhi/Downloads/LLM_Playground/Aditya_train.txt'}, score: 1.7037668228149414, embedding: vector of size 384)


In [9]:
from haystack import Document, Pipeline
# from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
from haystack.components.readers import ExtractiveReader


# document_store = InMemoryDocumentStore()
# document_store.write_documents(docs)

retriever = ChromaQueryTextRetriever(document_store = chroma_store)
#ExtractiveReader to extract answers from the relevant context
reader = ExtractiveReader(model="deepset/roberta-base-squad2-distilled")

extractive_qa_pipeline = Pipeline()
extractive_qa_pipeline.add_component(instance=retriever, name="retriever")
extractive_qa_pipeline.add_component(instance=reader, name="reader")

extractive_qa_pipeline.connect("retriever.documents", "reader.documents")

query = "Who are Aditya's friends?"
extractive_qa_pipeline.run(data={"retriever": {"query": query, "top_k": 3}, 
                                   "reader": {"query": query, "top_k": 2}})

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


{'reader': {'answers': [ExtractedAnswer(query="Who are Aditya's friends?", score=0.8281590938568115, data='SAS, Hunterr, MF, Rocco', document=Document(id=4131426fc6c30a306d6379a1d040425f2998e701287f2c255d53d6ad03d4c95d, content: 'Aditya Sugandhi's journey as a Software Engineer is characterized by a deep commitment to excellence...', meta: {'file_path': '/unity/f2/asugandhi/Downloads/LLM_Playground/Aditya_train.txt'}, score: 1.3431967496871948, embedding: vector of size 384), context=None, document_offset=ExtractedAnswer.Span(start=5015, end=5038), context_offset=None, meta={}),
   ExtractedAnswer(query="Who are Aditya's friends?", score=0.3937305212020874, data='\n', document=Document(id=1ab54edb49eae6dff289cbd2de1ab7426322d24f1be582d0dce19a209d36d164, content: 'Aditya Sugandhi's journey as a Software Engineer is characterized by a deep commitment to excellence...', meta: {'file_path': '/unity/f2/asugandhi/Downloads/LLM_Playground/Aditya_train.txt'}, score: 1.3431967496871948, embeddi